In [1]:
from daphne import daphne
import os

In [2]:
directory = 'programs/tests/deterministic'
os.chdir('/Users/gw/repos/prob_prog/hw/hw5/CS532-HW5/{}/'.format(directory))

In [87]:
i=1
exp = daphne(['desugar-hoppl', '-i', '../prob_prog/hw/hw5/CS532-HW5/programs/tests/hoppl-deterministic/test_{}.daphne'.format(i)])
exp

['fn',
 ['alpha'],
 ['+',
  ['push-address', 'alpha', 'addr0'],
  7,
  [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   8]]]

In [7]:
class Env():
    "An environment: a dict of {'var': val} pairs, with an outer Env."
    def __init__(self, parms=(), args=(), outer=None):
        self.data = pmap(zip(parms, args))
        self.outer = outer
        if outer is None:
            self.level = 0
        else:
            self.level = outer.level+1
    def __getitem__(self, item):
        return self.data[item]
    def find(self, var):
        "Find the innermost Env where var appears."
        if (var in self.data):
            return self
        else:
            if self.outer is not None:
                return self.outer.find(var)
            else:
                raise RuntimeError('var "{}" not found in outermost scope'.format(var))
    
    def print_env(self, print_lowest=False):
        print_limit = 1 if print_lowest == False else 0
        outer = self
        while outer is not None:
            if outer.level >= print_limit:
                print('Scope on level ', outer.level)
                if 'f' in outer:
                    print('Found f, ')
                    print(outer['f'].body)
                    print(outer['f'].parms)
                    print(outer['f'].env)
                print(outer,'\n')
            outer = outer.outer

In [12]:
import copy

class Procedure(object):
    "A user-defined Scheme procedure."
    def __init__(self, parms, body, env):
        self.parms, self.body, self.env = parms, body, env
    def __call__(self, *args): 
        new_env = copy.deepcopy(self.env)
        return eval(self.body, Env(self.parms, args, new_env)) # [0]

In [8]:
from pyrsistent import pmap

import torch
import torch.distributions as dist

In [133]:
def push_addr(alpha, value):
    return alpha + value

penv = {
    '+': torch.add,
    '*':torch.mul,
#     'push-address':push_addr
}

In [118]:
def standard_env():
    "An environment with some Scheme standard procedures."
    env = Env(penv.keys(), penv.values())
    return env

env = standard_env()
env.find('+')
#['+'](torch.tensor(0.),torch.tensor(1.))

In [135]:
def eval(x,env=standard_env(),sigma=None):

    print('x',x)
    if isinstance(x,str):
        return env.find(x)[x], sigma
    elif not isinstance(x,list):
        return torch.tensor(x),sigma
#     else:
#         assert False, 'case unmatched'
    
    op, param, *args = x
    
    if False:
        pass
    
    elif op == 'push-address':
        return '', sigma
    elif op == 'fn':
        print('args',args)
#         param, body = args
        body = args[0]
        return Procedure(param, body, env), sigma # has eval in it
        # param ['alpha', 'x']
        # body ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
        # env
    
    else:
        print('in else. x',x)
        proc, _ = eval(op,env,sigma)
        vals = ['']
        vals.extend([eval(arg, env, sigma)[0] for arg in args])
        print('vals',vals)

        if isinstance(proc, Procedure): # lambdas, not primitives
            print('in Procedure',proc, vals)
            r = proc(*vals)
        else:
            r = proc(*vals[1:]) # primitives
            print('in primitives',proc, vals)
            
        return r
    

# eval(['*','a0',3,3])  

# why vals only tensor(8.), where did push add go?

In [137]:
exp_sq_8 = [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   9]

eval(exp_sq_8)#(torch.tensor(8.))

x [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']], ['push-address', 'alpha', 'addr1'], 9]
in else. x [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']], ['push-address', 'alpha', 'addr1'], 9]
x ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
args [['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
x 9
vals ['', tensor(9)]
in Procedure <__main__.Procedure object at 0x12bf663a0> ['', tensor(9)]
x ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
in else. x ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']
x *
x x
x x
vals ['', tensor(9), tensor(9)]
in primitives <built-in method mul of type object at 0x10b12e1c0> ['', tensor(9), tensor(9)]


tensor(81)

In [88]:
exp

['fn',
 ['alpha'],
 ['+',
  ['push-address', 'alpha', 'addr0'],
  7,
  [['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']],
   ['push-address', 'alpha', 'addr1'],
   8]]]

In [95]:
exp_inner = ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
eval(exp_inner)#(torch.tensor(8.))

x ['fn', ['alpha', 'x'], ['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
args [['*', ['push-address', 'alpha', 'addr2'], 'x', 'x']]
